In [1]:
# GET stock data

In [2]:
import json
import requests
import pandas as pd
import time
from fake_useragent import UserAgent
from datetime import datetime
from sqlalchemy import create_engine
from collections import Counter
import mysql.connector
import setting

In [3]:
def mysql_to_df():
    mydb = mysql.connector.connect(
            host="192.168.22.110",
            user="root",
            passwd="Pn123456",
            database="Stock"
    )

    mycursor = mydb.cursor()
    mycursor.execute("SELECT * FROM stock_list")
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult,columns=["stock_number","stock_name","stock_createdate","stock_class","stock_type"])
    
    mycursor.close()
    mydb.close()
    
    return df


In [4]:
df = mysql_to_df()
stock_list = []
for i in df["stock_number"]:
    stock_list.append(str(i))

In [5]:

# stock = "2330"
# # local_date = datetime.now().strftime('%Y%m%d')
# local_date = "20200101"
# timestamp = str(int(time.time()))
# url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp


In [6]:
def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d

In [21]:
def Get_stock_data(url):
    
    ua = UserAgent()
    headers = {"User-Agent":ua.random}
    res = requests.get(url,headers=headers,timeout=10.0)
    json_stock = json.loads(res.content)
    print("status_code: ",res.status_code)
    result = []
    
    for i in range(len(json_stock["data"])):
        try:
            Date = datetime.strptime(transform_date(json_stock["data"][i][0]),"%Y/%m/%d")
            Transaction_count = int(json_stock["data"][i][1].replace(",",""))
            Transaction_amount = int(json_stock["data"][i][2].replace(",",""))
            Opening_price = float(json_stock["data"][i][3])
            Highest_price = float(json_stock["data"][i][4])
            Lowest_price = float(json_stock["data"][i][5])
            Closing_price = float(json_stock["data"][i][6])

            if json_stock["data"][i][7] == "X0.00":
                Price_spread = 0.00
            else:
                Price_spread = float(json_stock["data"][i][7])

            Number_of_transactions = int(json_stock["data"][i][8].replace(",",""))

            dict_result = {
                "Number":str(stock),
                "Date":Date,
                "Transaction_count":Transaction_count,
                "Transaction_amount":Transaction_amount,
                "Opening_price":Opening_price,
                "Highest_price":Highest_price,
                "Lowest_price":Lowest_price,
                "Closing_price":Closing_price,
                "Price_spread":Price_spread,
                "Number_of_transactions":Number_of_transactions

            }

            result.append(dict_result)
            
        except ValueError :
            pass 
        
    return result


In [8]:
user = "root"
passwd = "Pn123456"
ip = "192.168.22.110:3306"
db_name = "Stock"
table_name = "Stock_data"

def List_to_mysql(user,passwd,ip,db_name,table_name,result_list):

    engine = create_engine('mysql+mysqlconnector://'+ user +':'+ passwd +'@'+ip+'/'+ db_name +'?charset=utf8', encoding='utf-8')
    con = engine.connect()
    
    for item in result_list:
        df = pd.DataFrame(item, index=[0])
        try:
            df.to_sql(name=table_name,con=con,if_exists='append',index=False)

        except Exception as e:
            if 'PRIMARY' in str(e):
                pass

    con.close() 
    engine.dispose()

In [9]:
# result_list = Get_stock_data(url)
# List_to_mysql(user=user,passwd=passwd,ip=ip,db_name=db_name,table_name=table_name,result_list=result_list)

In [10]:
'''
CREATE TABLE IF NOT EXISTS `Stock_data` (
    `Number` CHAR(20) NOT NULL,
    `Date` DATE NOT NULL,
    `Transaction_count` INT NOT NULL,
    `Transaction_amount` INT NOT NULL,
    `Opening_price` FLOAT NOT NULL,
    `Highest_price` FLOAT NOT NULL,
    `Lowest_price` FLOAT NOT NULL,
    `Closing_price` FLOAT NOT NULL,
    `Price_spread` CHAR(10) NULL DEFAULT NULL,
    `Number_of_transactions` INT NOT NULL,
    PRIMARY KEY (`Number`, `Date`)
)
COLLATE='latin1_swedish_ci';

'''

"\nCREATE TABLE IF NOT EXISTS `Stock_data` (\n    `Number` CHAR(20) NOT NULL,\n    `Date` DATE NOT NULL,\n    `Transaction_count` INT NOT NULL,\n    `Transaction_amount` INT NOT NULL,\n    `Opening_price` FLOAT NOT NULL,\n    `Highest_price` FLOAT NOT NULL,\n    `Lowest_price` FLOAT NOT NULL,\n    `Closing_price` FLOAT NOT NULL,\n    `Price_spread` CHAR(10) NULL DEFAULT NULL,\n    `Number_of_transactions` INT NOT NULL,\n    PRIMARY KEY (`Number`, `Date`)\n)\nCOLLATE='latin1_swedish_ci';\n\n"

In [25]:
count = 0
for stock in stock_list[12:]:
    for m in range(1,7):
        try :
            local_date = "20200"+str(m)+"01"
            timestamp = str(int(time.time()))
            url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp
            result_list = Get_stock_data(url)
            List_to_mysql(user=user,passwd=passwd,ip=ip,db_name=db_name,table_name=table_name,result_list=result_list)
            count += 1
            print("股票代號: ",stock,"月份: ",local_date,"請求次數: ",count)
            time.sleep(3)
        except:
            print("股票代號: ",stock,"月份: ",local_date,"請求次數: ",count," timeout")
        


status_code:  200
股票代號:  1216 月份:  20200101 請求次數:  1
status_code:  200
股票代號:  1216 月份:  20200201 請求次數:  2
status_code:  200
股票代號:  1216 月份:  20200301 請求次數:  3
status_code:  200
股票代號:  1216 月份:  20200401 請求次數:  4
status_code:  200
股票代號:  1216 月份:  20200501 請求次數:  5
status_code:  200
股票代號:  1216 月份:  20200601 請求次數:  6
status_code:  200
股票代號:  1217 月份:  20200101 請求次數:  7
status_code:  200
股票代號:  1217 月份:  20200201 請求次數:  8
股票代號:  1217 月份:  20200301 請求次數:  8  timeout
status_code:  200
股票代號:  1217 月份:  20200401 請求次數:  9
status_code:  200
股票代號:  1217 月份:  20200501 請求次數:  10
status_code:  200
股票代號:  1217 月份:  20200601 請求次數:  11
status_code:  200
股票代號:  1218 月份:  20200101 請求次數:  12
status_code:  200
股票代號:  1218 月份:  20200201 請求次數:  13
status_code:  200
股票代號:  1218 月份:  20200301 請求次數:  14
status_code:  200
股票代號:  1218 月份:  20200401 請求次數:  15
status_code:  200
股票代號:  1218 月份:  20200501 請求次數:  16
status_code:  200
股票代號:  1218 月份:  20200601 請求次數:  17
status_code:  200
股票代號:  1219 月份:  20200101 請求次

status_code:  200
股票代號:  1314 月份:  20200301 請求次數:  152
status_code:  200
股票代號:  1314 月份:  20200401 請求次數:  153
status_code:  200
股票代號:  1314 月份:  20200501 請求次數:  154
status_code:  200
股票代號:  1314 月份:  20200601 請求次數:  155
status_code:  200
股票代號:  1315 月份:  20200101 請求次數:  156
status_code:  200
股票代號:  1315 月份:  20200201 請求次數:  157
status_code:  200
股票代號:  1315 月份:  20200301 請求次數:  158
status_code:  200
股票代號:  1315 月份:  20200401 請求次數:  159
status_code:  200
股票代號:  1315 月份:  20200501 請求次數:  160
status_code:  200
股票代號:  1315 月份:  20200601 請求次數:  161
status_code:  200
股票代號:  1316 月份:  20200101 請求次數:  162
status_code:  200
股票代號:  1316 月份:  20200201 請求次數:  163
status_code:  200
股票代號:  1316 月份:  20200301 請求次數:  164
status_code:  200
股票代號:  1316 月份:  20200401 請求次數:  165
status_code:  200
股票代號:  1316 月份:  20200501 請求次數:  166
status_code:  200
股票代號:  1316 月份:  20200601 請求次數:  167
status_code:  200
股票代號:  1319 月份:  20200101 請求次數:  168
status_code:  200
股票代號:  1319 月份:  20200201 請求次數:  169
status_cod

status_code:  200
股票代號:  1434 月份:  20200301 請求次數:  299
status_code:  200
股票代號:  1434 月份:  20200401 請求次數:  300
status_code:  200
股票代號:  1434 月份:  20200501 請求次數:  301
status_code:  200
股票代號:  1434 月份:  20200601 請求次數:  302
status_code:  200
股票代號:  1435 月份:  20200101 請求次數:  303
status_code:  200
股票代號:  1435 月份:  20200201 請求次數:  304
status_code:  200
股票代號:  1435 月份:  20200301 請求次數:  305
status_code:  200
股票代號:  1435 月份:  20200401 請求次數:  306
status_code:  200
股票代號:  1435 月份:  20200501 請求次數:  307
status_code:  200
股票代號:  1435 月份:  20200601 請求次數:  308
status_code:  200
股票代號:  1436 月份:  20200101 請求次數:  309
status_code:  200
股票代號:  1436 月份:  20200201 請求次數:  310
status_code:  200
股票代號:  1436 月份:  20200301 請求次數:  311
status_code:  200
股票代號:  1436 月份:  20200401 請求次數:  312
status_code:  200
股票代號:  1436 月份:  20200501 請求次數:  313
status_code:  200
股票代號:  1436 月份:  20200601 請求次數:  314
status_code:  200
股票代號:  1437 月份:  20200101 請求次數:  315
status_code:  200
股票代號:  1437 月份:  20200201 請求次數:  316
status_cod

status_code:  200
股票代號:  1464 月份:  20200401 請求次數:  440
status_code:  200
股票代號:  1464 月份:  20200501 請求次數:  441
status_code:  200
股票代號:  1464 月份:  20200601 請求次數:  442
status_code:  200
股票代號:  1465 月份:  20200101 請求次數:  443
status_code:  200
股票代號:  1465 月份:  20200201 請求次數:  444
status_code:  200
股票代號:  1465 月份:  20200301 請求次數:  445
status_code:  200
股票代號:  1465 月份:  20200401 請求次數:  446
status_code:  200
股票代號:  1465 月份:  20200501 請求次數:  447
status_code:  200
股票代號:  1465 月份:  20200601 請求次數:  448
status_code:  200
股票代號:  1466 月份:  20200101 請求次數:  449
status_code:  200
股票代號:  1466 月份:  20200201 請求次數:  450
status_code:  200
股票代號:  1466 月份:  20200301 請求次數:  451
status_code:  200
股票代號:  1466 月份:  20200401 請求次數:  452
status_code:  200
股票代號:  1466 月份:  20200501 請求次數:  453
status_code:  200
股票代號:  1466 月份:  20200601 請求次數:  454
status_code:  200
股票代號:  1467 月份:  20200101 請求次數:  455
status_code:  200
股票代號:  1467 月份:  20200201 請求次數:  456
status_code:  200
股票代號:  1467 月份:  20200301 請求次數:  457
status_cod

status_code:  200
股票代號:  1522 月份:  20200401 請求次數:  585
status_code:  200
股票代號:  1522 月份:  20200501 請求次數:  586
status_code:  200
股票代號:  1522 月份:  20200601 請求次數:  587
status_code:  200
股票代號:  1524 月份:  20200101 請求次數:  588
status_code:  200
股票代號:  1524 月份:  20200201 請求次數:  589
status_code:  200
股票代號:  1524 月份:  20200301 請求次數:  590
status_code:  200
股票代號:  1524 月份:  20200401 請求次數:  591
status_code:  200
股票代號:  1524 月份:  20200501 請求次數:  592
status_code:  200
股票代號:  1524 月份:  20200601 請求次數:  593
status_code:  200
股票代號:  1525 月份:  20200101 請求次數:  594
status_code:  200
股票代號:  1525 月份:  20200201 請求次數:  595
status_code:  200
股票代號:  1525 月份:  20200301 請求次數:  596
status_code:  200
股票代號:  1525 月份:  20200401 請求次數:  597
status_code:  200
股票代號:  1525 月份:  20200501 請求次數:  598
status_code:  200
股票代號:  1525 月份:  20200601 請求次數:  599
status_code:  200
股票代號:  1526 月份:  20200101 請求次數:  600
status_code:  200
股票代號:  1526 月份:  20200201 請求次數:  601
status_code:  200
股票代號:  1526 月份:  20200301 請求次數:  602
status_cod

...

status_code:  200
股票代號:  6541 月份:  20200501 請求次數:  4674
status_code:  200
股票代號:  6541 月份:  20200601 請求次數:  4675
status_code:  200
股票代號:  6552 月份:  20200101 請求次數:  4676
status_code:  200
股票代號:  6552 月份:  20200201 請求次數:  4677
股票代號:  6552 月份:  20200301 請求次數:  4677  timeout
status_code:  200
股票代號:  6552 月份:  20200401 請求次數:  4678
status_code:  200
股票代號:  6552 月份:  20200501 請求次數:  4679
status_code:  200
股票代號:  6552 月份:  20200601 請求次數:  4680
status_code:  200
股票代號:  6558 月份:  20200101 請求次數:  4681
status_code:  200
股票代號:  6558 月份:  20200201 請求次數:  4682
status_code:  200
股票代號:  6558 月份:  20200301 請求次數:  4683
status_code:  200
股票代號:  6558 月份:  20200401 請求次數:  4684
status_code:  200
股票代號:  6558 月份:  20200501 請求次數:  4685
status_code:  200
股票代號:  6558 月份:  20200601 請求次數:  4686
status_code:  200
股票代號:  6573 月份:  20200101 請求次數:  4687
status_code:  200
股票代號:  6573 月份:  20200201 請求次數:  4688
status_code:  200
股票代號:  6573 月份:  20200301 請求次數:  4689
status_code:  200
股票代號:  6573 月份:  20200401 請求次數:  4690
s

status_code:  200
股票代號:  8016 月份:  20200301 請求次數:  4805
status_code:  200
股票代號:  8016 月份:  20200401 請求次數:  4806
status_code:  200
股票代號:  8016 月份:  20200501 請求次數:  4807
status_code:  200
股票代號:  8016 月份:  20200601 請求次數:  4808
status_code:  200
股票代號:  8021 月份:  20200101 請求次數:  4809
status_code:  200
股票代號:  8021 月份:  20200201 請求次數:  4810
status_code:  200
股票代號:  8021 月份:  20200301 請求次數:  4811
status_code:  200
股票代號:  8021 月份:  20200401 請求次數:  4812
status_code:  200
股票代號:  8021 月份:  20200501 請求次數:  4813
status_code:  200
股票代號:  8021 月份:  20200601 請求次數:  4814
status_code:  200
股票代號:  8028 月份:  20200101 請求次數:  4815
status_code:  200
股票代號:  8028 月份:  20200201 請求次數:  4816
status_code:  200
股票代號:  8028 月份:  20200301 請求次數:  4817
status_code:  200
股票代號:  8028 月份:  20200401 請求次數:  4818
status_code:  200
股票代號:  8028 月份:  20200501 請求次數:  4819
status_code:  200
股票代號:  8028 月份:  20200601 請求次數:  4820
status_code:  200
股票代號:  8033 月份:  20200101 請求次數:  4821
status_code:  200
股票代號:  8033 月份:  20200201 請求次數

status_code:  200
股票代號:  8261 月份:  20200101 請求次數:  4944
status_code:  200
股票代號:  8261 月份:  20200201 請求次數:  4945
status_code:  200
股票代號:  8261 月份:  20200301 請求次數:  4946
status_code:  200
股票代號:  8261 月份:  20200401 請求次數:  4947
status_code:  200
股票代號:  8261 月份:  20200501 請求次數:  4948
status_code:  200
股票代號:  8261 月份:  20200601 請求次數:  4949
status_code:  200
股票代號:  8271 月份:  20200101 請求次數:  4950
status_code:  200
股票代號:  8271 月份:  20200201 請求次數:  4951
status_code:  200
股票代號:  8271 月份:  20200301 請求次數:  4952
status_code:  200
股票代號:  8271 月份:  20200401 請求次數:  4953
status_code:  200
股票代號:  8271 月份:  20200501 請求次數:  4954
status_code:  200
股票代號:  8271 月份:  20200601 請求次數:  4955
status_code:  200
股票代號:  8341 月份:  20200101 請求次數:  4956
status_code:  200
股票代號:  8341 月份:  20200201 請求次數:  4957
status_code:  200
股票代號:  8341 月份:  20200301 請求次數:  4958
status_code:  200
股票代號:  8341 月份:  20200401 請求次數:  4959
status_code:  200
股票代號:  8341 月份:  20200501 請求次數:  4960
status_code:  200
股票代號:  8341 月份:  20200601 請求次數

status_code:  200
股票代號:  8497 月份:  20200501 請求次數:  5084
status_code:  200
股票代號:  8497 月份:  20200601 請求次數:  5085
status_code:  200
股票代號:  8499 月份:  20200101 請求次數:  5086
status_code:  200
股票代號:  8499 月份:  20200201 請求次數:  5087
status_code:  200
股票代號:  8499 月份:  20200301 請求次數:  5088
status_code:  200
股票代號:  8499 月份:  20200401 請求次數:  5089
status_code:  200
股票代號:  8499 月份:  20200501 請求次數:  5090
status_code:  200
股票代號:  8499 月份:  20200601 請求次數:  5091
股票代號:  8926 月份:  20200101 請求次數:  5091  timeout
status_code:  200
股票代號:  8926 月份:  20200201 請求次數:  5092
status_code:  200
股票代號:  8926 月份:  20200301 請求次數:  5093
status_code:  200
股票代號:  8926 月份:  20200401 請求次數:  5094
status_code:  200
股票代號:  8926 月份:  20200501 請求次數:  5095
status_code:  200
股票代號:  8926 月份:  20200601 請求次數:  5096
status_code:  200
股票代號:  8940 月份:  20200101 請求次數:  5097
status_code:  200
股票代號:  8940 月份:  20200201 請求次數:  5098
status_code:  200
股票代號:  8940 月份:  20200301 請求次數:  5099
status_code:  200
股票代號:  8940 月份:  20200401 請求次數:  5100
s

股票代號:  9929 月份:  20200301 請求次數:  5219
status_code:  200
股票代號:  9929 月份:  20200401 請求次數:  5220
status_code:  200
股票代號:  9929 月份:  20200501 請求次數:  5221
status_code:  200
股票代號:  9929 月份:  20200601 請求次數:  5222
status_code:  200
股票代號:  9930 月份:  20200101 請求次數:  5223
status_code:  200
股票代號:  9930 月份:  20200201 請求次數:  5224
status_code:  200
股票代號:  9930 月份:  20200301 請求次數:  5225
status_code:  200
股票代號:  9930 月份:  20200401 請求次數:  5226
status_code:  200
股票代號:  9930 月份:  20200501 請求次數:  5227
status_code:  200
股票代號:  9930 月份:  20200601 請求次數:  5228
status_code:  200
股票代號:  9931 月份:  20200101 請求次數:  5229
status_code:  200
股票代號:  9931 月份:  20200201 請求次數:  5230
status_code:  200
股票代號:  9931 月份:  20200301 請求次數:  5231
status_code:  200
股票代號:  9931 月份:  20200401 請求次數:  5232
status_code:  200
股票代號:  9931 月份:  20200501 請求次數:  5233
status_code:  200
股票代號:  9931 月份:  20200601 請求次數:  5234
status_code:  200
股票代號:  9933 月份:  20200101 請求次數:  5235
status_code:  200
股票代號:  9933 月份:  20200201 請求次數:  5236
status_cod

In [ ]:
if res.status_code == requests.codes.ok:
    print("OK")

In [23]:
stock_list.index('1216')

12

In [27]:
stock = "1217 "
local_date = "20200301"
timestamp = str(int(time.time()))
url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp
result_list = Get_stock_data(url)
List_to_mysql(user=user,passwd=passwd,ip=ip,db_name=db_name,table_name=table_name,result_list=result_list)

status_code:  200
